# Evaulate Distilbert TopicChooser Model
### 25 Nov 2020

## I. Evaluation Parameters

In [1]:
# Set Evaulation Parameters
MODEL_NAME = "distilbert-base-uncased"
MODEL_FILE ="saved_model_epoch5_20201126_1725.tar"
VAL_DATASET = "val_dataset_25Nov_source_distilbert.pickle"
EVAL_BATCH_SIZE = 64
DESCRIPTION = "Distilbert Topic Chooser Model trained on gossipcop and politifact"

## II. Setup and Load Model

In [2]:
import pickle
import sys

import json
import pandas as pd
import pickle
import sklearn.metrics
import tqdm
import torch
import transformers as hft

sys.path.insert(0, "/home/jupyter")
import util.log
import util.data

In [3]:
# Load evaluation data
with open(VAL_DATASET, "rb") as vfile:
    val_dataset = pickle.load(vfile)

In [4]:
# Load saved model
checkpoint = torch.load(MODEL_FILE)
model = (hft.DistilBertForSequenceClassification
         .from_pretrained(MODEL_NAME))
model.load_state_dict(checkpoint["model"])

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

<All keys matched successfully>

In [5]:
# Move model to GPU if available
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
model.to(device)
model.eval();
device

device(type='cuda')

## III. Evaluation Loop and Results Dataframe

In [6]:
# Evaulate model
eval_loader = torch.utils.data.DataLoader(val_dataset,
                                batch_size=EVAL_BATCH_SIZE,
                                shuffle=False)
labels = []
preds = []
logits = []
probs = []
sources = []
article_tokens = []
file_names = []

softmax = torch.nn.Softmax(dim=1)

with torch.no_grad():
    for art_num, article in tqdm.tqdm(enumerate(eval_loader)):
        input_ids = article["input_ids"].to(device)
        attent_mask = article["attention_mask"].to(device)
        output = model(input_ids,
                       attention_mask=attent_mask,
                       output_hidden_states=False,
                       output_attentions=False)
        prob = softmax(output[0].detach().cpu()).numpy()
        logit = output[0].detach().cpu().numpy()
        pred = [0 if lgt[0] > lgt[1] else 1 for lgt in logit]
        label = article["labels"].numpy()
        labels.extend(label)
        preds.extend(pred)
        logits.extend(logit)
        probs.extend(prob)
        article_tokens.extend(article["article_tokens"].numpy())
        sources.extend(article["sources"])
        file_names.extend(article["file_names"])

0it [00:00, ?it/s]/home/jupyter/util/data.py:435: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
58it [00:31,  1.87it/s]


In [13]:
eval_results = pd.DataFrame({"Article": list(range(len(labels))),
                             "Source": sources,
                             "Token_Length": article_tokens,
                             "Predictions": preds,
                             "Labels": labels,
                             "Logit_gossipcop(0)": [x[0] for x in logits],
                             "Logit_politifact(1)": [x[1] for x in logits],
                             "Prob_gc(0)": [x[0] for x in probs],
                             "Prob_pf(1)": [x[1] for x in probs],
                             "File_Name": file_names})

eval_results

,Article,Source,Token_Length,Predictions,Labels,Logit_gossipcop(0),Logit_politifact(1),Prob_gc(0),Prob_pf(1),File_Name
0,0,gossipcop,1366,0,0,4.818519,-5.395754,0.999963,0.000037,gossipcop-848731
1,1,gossipcop,509,0,0,4.819828,-5.409225,0.999964,0.000036,gossipcop-944840
2,2,gossipcop,298,0,0,4.820958,-5.412676,0.999964,0.000036,gossipcop-1267637959
3,3,gossipcop,1493,0,0,4.764092,-5.395077,0.999961,0.000039,gossipcop-909119
4,4,gossipcop,151,0,0,4.761987,-5.337341,0.999959,0.000041,gossipcop-866623
...,...,...,...,...,...,...,...,...,...,...
3658,3658,gossipcop,670,0,0,4.820902,-5.429477,0.999965,0.000035,gossipcop-6231788357
3659,3659,gossipcop,218,0,0,4.796435,-5.386364,0.999962,0.000038,gossipcop-896655
3660,3660,gossipcop,10233,0,0,2.757887,-3.260963,0.997573,0.002427,gossipcop-856760
3661,3661,gossipcop,459,0,0,4.786703,-5.391926,0.999962,0.000038,gossipcop-854426


In [14]:
# Save evaluation data to csv file
eval_results.to_csv("eval_data_" + MODEL_FILE[12:-4] + ".csv")

## IV. Overall Metrics

In [27]:
# Overall metrics
precision, recall, f1, _ = sklearn.metrics.precision_recall_fscore_support(
    eval_results.Labels, eval_results.Predictions, average="binary")
accuracy = sklearn.metrics.accuracy_score(eval_results.Labels, eval_results.Predictions)

metrics = {"description": DESCRIPTION,
           "eval_notes": "Evaluating both gossipcop and poltifact data",
           "model": MODEL_FILE,
           "epochs": checkpoint["epoch"],
           "batch_size": EVAL_BATCH_SIZE,
           "Date_evaluated": "25Nov2020",
           "train_data": "train_dataset_25Nov_source_distilbert.pickle",
           "val_data": "eval_data_" + MODEL_FILE[12:-4] + ".csv",
           "eval_data": VAL_DATASET,
           "accuracy": accuracy,
           "precision": precision,
           "recall": recall,
           "f1": f1}
metrics

{'description': 'Distilbert Topic Chooser Model trained on gossipcop and politifact',
 'eval_notes': 'Evaluating both gossipcop and poltifact data',
 'model': 'saved_model_epoch5_20201126_1725.tar',
 'epochs': 5,
 'batch_size': 64,
 'Date_evaluated': '25Nov2020',
 'train_data': 'train_dataset_25Nov_source_distilbert.pickle',
 'val_data': 'eval_data_epoch5_20201126_1725.csv',
 'eval_data': 'val_dataset_25Nov_source_distilbert.pickle',
 'accuracy': 0.9773409773409774,
 'precision': 0.9655172413793104,
 'recall': 0.40875912408759124,
 'f1': 0.5743589743589743}

In [10]:
# Save metrics to JSON text file.
with open("eval_metrics_" + MODEL_FILE[12:-4] + ".json", "wt") as jfile:
    json.dump(metrics, jfile)

In [38]:
cm = pd.DataFrame(sklearn.metrics.confusion_matrix(eval_results.Labels, eval_results.Predictions))
label_titles = {0: "gossip", 1: "political"}
cm.rename(index=label_titles, columns=label_titles, inplace=True)
cm

,gossip,political
gossip,3524,2
political,81,56
